In [1]:
!pip install kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yyt0817","key":"93acccd48fc47c9f13117065c32c5c6d"}'}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json  ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d wilmerarltstrmberg/recipe-dataset-over-2m

 99% 632M/635M [00:12<00:00, 81.3MB/s]
100% 635M/635M [00:12<00:00, 52.9MB/s]


In [6]:
!unzip -q recipe-dataset-over-2m.zip

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
data = pd.read_csv("recipes_data.csv")

In [9]:
data.head()

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com


In [10]:
from numpy.random import default_rng
data = data[0:5000]

In [11]:
#get rid of useless attributes: site, source, link
data = data.drop(['site', 'source', 'link'], axis=1)

In [12]:
data.head()

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""bite size shredded rice biscuits"", ""vanilla""..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""cream of mushroom soup"", ""beef"", ""sour cream..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""pepper"", ""cream cheese"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken gravy"", ""cream of mushroom soup"", ""c..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""graham cracker crumbs"", ""powdered sugar"", ""p..."


In [13]:
import ast
# Convert the string representation of the list into an actual list
data['directions'] = data['directions'].apply(ast.literal_eval)
data['NER'] = data['NER'].apply(ast.literal_eval)
data['ingredients'] = data['ingredients'].apply(ast.literal_eval)

In [14]:
#get all unique ingredients
ingredients_vocab = set(ingredient for sublist in data['NER'] for ingredient in sublist)

In [15]:
def preprocess(strings_list):
    return [string.replace(", ", " , ").replace(".", " . ") for string in strings_list]

In [16]:
data['directions'] = data['directions'].apply(preprocess)
data['NER'] = data['NER'].apply(preprocess)
data['ingredients'] = data['ingredients'].apply(preprocess)

In [17]:
ingredients_list = list(data['ingredients'])
directions_list = list(data['directions'])
title_list = list(data['title'])
ner_list = list(data['NER'])

In [18]:
def preprocess_dataset(ingredients_list, directions_list, title_list, ner_list):
    # Initialize an empty list to store preprocessed data
    question = []
    answer = []

    # Iterate over each recipe in the dataset
    for ingredients, directions, title, ner in zip(ingredients_list, directions_list, title_list, ner_list):
        # Combine ingredients and directions into a single string with spaces between words/symbols
        ingredients_text = ' , '.join(ingredients)
        directions_text = ' '.join(directions)
        ner_text = ' '.join(ner)
        question.append(f"I have {ner_text} . Give me some cooking ideas .")
        answer.append(f"you can make {title} with {ingredients_text} . Here's the instruction : {directions_text}")

    return question, answer

In [19]:
question, answer = preprocess_dataset(ingredients_list, directions_list, title_list, ner_list)

In [20]:
df = pd.DataFrame(list(zip(question, answer)),
               columns =['question', 'answer'])

In [21]:
df.head()

,question,answer
0,I have bite size shredded rice biscuits vanill...,you can make No-Bake Nut Cookies with 1 c . f...
1,I have cream of mushroom soup beef sour cream ...,you can make Jewell Ball'S Chicken with 1 smal...
2,I have frozen corn pepper cream cheese garlic ...,you can make Creamy Corn with 2 (16 oz . ) pkg...
3,I have chicken gravy cream of mushroom soup ch...,you can make Chicken Funny with 1 large whole ...
4,I have graham cracker crumbs powdered sugar pe...,you can make Reeses Cups(Candy) with 1 c . ...


In [22]:
df.to_csv('/content/drive/My Drive/recipe_dataset_v2_5000.csv', index=False)